In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Subset, ConcatDataset
from randaugment import RandAugmentMC



CUDA = True
CUDA = CUDA and torch.cuda.is_available()
seed = 42
print("PyTorch version: {}".format(torch.__version__))
if CUDA:
    print("CUDA version: {}\n".format(torch.version.cuda))

if CUDA:
    torch.cuda.manual_seed(seed)
device = torch.device("cuda:0" if CUDA else "cpu")
cudnn.benchmark = True

2024-04-23 17:46:04.743882: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-23 17:46:04.747840: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 17:46:04.799666: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 17:46:05.639799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


PyTorch version: 1.13.1
CUDA version: 11.7



In [3]:
# Load the dataset
dataset, info = tfds.load('colorectal_histology', with_info=True, as_supervised=True)
dataset = dataset['train'].batch(len(dataset['train']))

2024-04-23 17:46:09.557538: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 17:46:09.561272: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# Transform the dataset into pytorch
for images, labels in dataset:
    images_tensor = torch.tensor(images.numpy(), dtype=torch.float)
    images_tensor = images_tensor.permute(0, 3, 1, 2)
    labels_tensor = torch.tensor(labels.numpy(), dtype=torch.long)

2024-04-23 17:46:14.541408: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
class ColorectalHistDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx].clone().detach()
        label = self.labels[idx].clone().detach()

        if self.transform:
            image = self.transform(image)

        return image, label


In [6]:
class ColorectalHistDatasetSemi(Dataset):
    def __init__(self, images, labels, transform_w=None, transform_s=None):
        self.images = images
        self.labels = labels
        self.transform_w = transform_w
        self.transform_s = transform_s

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx].clone().detach()
        label = self.labels[idx].clone().detach()

        if self.transform_w:
            image_w = self.transform_w(image)
        else:
            image_w = image

        if self.transform_s:
            image_s = self.transform_s(image)
        else:
            image_s = image


        return image_w, image_s, label

In [7]:
def stratified_split(dataset, test_size=0.2):

    labels = np.array([ins[-1] for ins in dataset])

    # Indices for each class
    class_indices = [np.where(labels == class_label)[0] for class_label in np.unique(labels)]

    # Split each class's indices into train and test
    train_indices, test_indices = [], []
    for indices in class_indices:
        np.random.shuffle(indices)
        split = int(np.floor(test_size * len(indices)))
        train_indices.extend(indices[split:])
        test_indices.extend(indices[:split])

    # Create subset for train and test
    train_subset = Subset(dataset, train_indices)
    test_subset = Subset(dataset, test_indices)

    return train_subset, test_subset


In [8]:
# Transformation for supervised learning
transform = transforms.Compose([

    transforms.ToPILImage(),  # Convert numpy array to PIL Image to apply transforms
    transforms.RandomHorizontalFlip(p=0.5),  # Apply horizontal flip with 50% probability
    transforms.RandomVerticalFlip(p=0.5),    # Apply vertical flip with 50% probability
    # Add any additional transformations here (e.g., resizing, normalization)
    transforms.RandomResizedCrop(size=(150, 150), antialias=True),
    transforms.ToTensor(),  # Convert PIL Image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ColorectalHistDataset(images_tensor, labels_tensor, transform)
train_subset, test_subset = stratified_split(dataset)

In [9]:
# Transformation for semi-supervised learning
transform_w = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array to PIL Image to apply transforms
    transforms.RandomHorizontalFlip(p=0.5),  # Apply horizontal flip with 50% probability
    transforms.RandomVerticalFlip(p=0.5),    # Apply vertical flip with 50% probability
    transforms.ToTensor(),  # Convert PIL Image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_s = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array to PIL Image to apply transforms
    transforms.RandomHorizontalFlip(p=0.5),  # Apply horizontal flip with 50% probability
    transforms.RandomVerticalFlip(p=0.5),    # Apply vertical flip with 50% probability
    RandAugmentMC(n=2, m=10),
    transforms.ToTensor(),  # Convert PIL Image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [15]:
# The base learner

class CRCClassifier(nn.Module):
    def __init__(self, num_classes=8):
        super(CRCClassifier, self).__init__()
        self.num_classes = num_classes
        resnet18 = models.resnet18(pretrained=True)
        resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
        self.conv_layers = resnet18

    def forward(self, x):
        return self.conv_layers(x)

model = CRCClassifier().to(device)

In [13]:
torch.randn(10).cuda()

tensor([-0.1947, -0.8845,  0.7924,  1.2290,  0.5628, -0.1965,  0.4304, -0.5121,
        -2.1681,  1.2008], device='cuda:0')

In [16]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [17]:
set_seed(seed)

In [18]:
# Settings for training
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
num_epochs = 30
eval_step = 200
threshold = 0.95
batch_size = 40
ratio = 5
lambda_u = 1

In [19]:
# Labeled instances selection
dataset = ColorectalHistDatasetSemi(images_tensor, labels_tensor, transform_w, transform_s)
train_subset, test_subset = stratified_split(dataset)
unlabeled_subset, labeled_subset = stratified_split(train_subset, test_size=0.02)
labeled_loader = DataLoader(labeled_subset, batch_size=batch_size, shuffle=True, num_workers=2)
unlabeled_loader = DataLoader(unlabeled_subset, batch_size=batch_size*ratio, shuffle=True, num_workers=2)
val_loader = DataLoader(test_subset, batch_size=40, shuffle=False)


In [20]:
def interleave(x, batch_size):
    s = list(x.shape)
    return x.reshape([-1, batch_size] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])


def de_interleave(x, size):
    s = list(x.shape)
    return x.reshape([size, -1] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])

def train_fixmatch(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u):
    model.train()
    labeled_iter = iter(labeled_loader)
    unlabeled_iter = iter(unlabeled_loader)
    running_loss = 0.0
    for epoch in range(num_epochs):
        loss_s = 0.0
        loss_u = 0.0
        for batch_idx in range(eval_step):
            try:
                inputs_x, _, targets_x = next(labeled_iter)
            except StopIteration:
                labeled_iter = iter(labeled_loader)
                inputs_x, _, targets_x = next(labeled_iter)

            try:
                inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)
            except StopIteration:
                unlabeled_iter = iter(unlabeled_loader)
                inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)

            inputs_x = inputs_x.to(device)
            targets_x = targets_x.to(device)
            inputs_u_w = inputs_u_w.to(device)
            inputs_u_s = inputs_u_s.to(device)

            # Forward pass
            optimizer.zero_grad()
            inputs = interleave(torch.cat((inputs_x, inputs_u_w, inputs_u_s)), ratio*2+1).to(device)
            logits = model(inputs)
            logits = de_interleave(logits, ratio*2+1)
            logits_x = logits[:batch_size]
            logits_u_w, logits_u_s = logits[batch_size:].chunk(2)
            del logits

            # Compute the loss for labeled data
            Lx = criterion(logits_x, targets_x)
            loss_s += Lx.item() * inputs_x.size(0)

            # Compute the pseudo-labels and loss for unlabeled data
            probs = torch.softmax(logits_u_w, dim=-1)
            max_probs, targets_u = torch.max(probs, dim=-1)
            mask = max_probs.ge(threshold).bool()

            Lu = (F.cross_entropy(logits_u_s, targets_u, reduction='none') * mask).mean()

            loss_u += Lu.item() * mask.sum()

            # Total loss
            loss = Lx + lambda_u * Lu

            # Backward and optimize
            loss.backward()
            optimizer.step()
            model.zero_grad()

        loss_s = loss_s / (eval_step * batch_size)
        loss_u = loss_u / (eval_step * batch_size * ratio)

        print(f'Supervised Loss: {loss_s}, Unsupervised Loss: {loss_u}')

        # Validation phase
        model.eval()  # Set model to evaluate mode
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs_w, inputs_s, labels in val_loader:
                inputs = inputs_w.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(test_subset)
        val_acc = val_corrects.double() / len(test_subset)
        print(f'Validation Loss: {val_loss}, Acc: {val_acc}')

In [21]:
def train_fixmatch_multistep(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1): 
    model.train()
    labeled_iter = iter(labeled_loader)
    unlabeled_iter = iter(unlabeled_loader)
    best_acc = 0.0
    for epoch in range(num_epochs):
        loss_s = 0.0
        loss_u = 0.0
        for batch_idx in range(eval_step):
            try:
                inputs_x, _, targets_x = next(labeled_iter)
            except StopIteration:
                labeled_iter = iter(labeled_loader)
                inputs_x, _, targets_x = next(labeled_iter)

            try:
                inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)
                if len(inputs_u_w) < batch_size*ratio:
                    unlabeled_iter = iter(unlabeled_loader)
                    inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)
            except StopIteration:
                unlabeled_iter = iter(unlabeled_loader)
                inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)

            inputs_x = inputs_x.to(device)
            targets_x = targets_x.to(device)
            inputs_u_w = inputs_u_w.to(device)
            inputs_u_s = inputs_u_s.to(device)
            # print(inputs_x.shape[0], inputs_u_w.shape[0], inputs_u_s.shape[0])

            # Forward pass
            optimizer.zero_grad()
            inputs = interleave(torch.cat((inputs_x, inputs_u_w, inputs_u_s)), ratio*2+1).to(device)
            logits = model(inputs)
            logits = de_interleave(logits, ratio*2+1)
            logits_x = logits[:batch_size]
            logits_u_w, logits_u_s = logits[batch_size:].chunk(2)
            del logits

            # Compute the loss for labeled data
            Lx = criterion(logits_x, targets_x)
            loss_s += Lx.item() * inputs_x.size(0)

            # Compute the pseudo-labels and loss for unlabeled data
            probs = torch.softmax(logits_u_w, dim=-1)
            max_probs, targets_u = torch.max(probs, dim=-1)
            mask = max_probs.ge(threshold).bool()

            Lu = (F.cross_entropy(logits_u_s, targets_u, reduction='none') * mask).mean()

            loss_u += Lu.item() * mask.sum()

            # Total loss
            loss = Lx + lambda_u * Lu

            # Backward and optimize
            loss.backward()
            optimizer.step()
            model.zero_grad()

        loss_s = loss_s / (eval_step * batch_size)
        loss_u = loss_u / (eval_step * batch_size * ratio)

        print(f'Supervised Loss: {loss_s}, Unsupervised Loss: {loss_u}')

        # Validation phase
        model.eval()  # Set model to evaluate mode
        val_loss = 0.0
        val_corrects = 0
        
        with torch.no_grad():
            for inputs_w, inputs_s, labels in val_loader:
                inputs = inputs_w.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(test_subset)
        val_acc = val_corrects.double() / len(test_subset)
        print(f'Validation Loss: {val_loss}, Acc: {val_acc}')
        if val_acc > best_acc:
            best_acc = val_acc
        
    return best_acc

In [20]:
def train_fixmatch_active(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1): 
    model.train()
    labeled_iter = iter(labeled_loader)
    unlabeled_iter = iter(unlabeled_loader)
    best_acc = 0.0
    for epoch in range(num_epochs):
        loss_s = 0.0
        loss_u = 0.0
        for batch_idx in range(eval_step):
            try:
                inputs_x, _, targets_x = next(labeled_iter)
            except StopIteration:
                labeled_iter = iter(labeled_loader)
                inputs_x, _, targets_x = next(labeled_iter)

            try:
                inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)
                if len(inputs_u_w) < batch_size*ratio:
                    unlabeled_iter = iter(unlabeled_loader)
                    inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)
            except StopIteration:
                unlabeled_iter = iter(unlabeled_loader)
                inputs_u_w, inputs_u_s, _ = next(unlabeled_iter)

            inputs_x = inputs_x.to(device)
            targets_x = targets_x.to(device)
            inputs_u_w = inputs_u_w.to(device)
            inputs_u_s = inputs_u_s.to(device)
            # print(inputs_x.shape[0], inputs_u_w.shape[0], inputs_u_s.shape[0])

            # Forward pass
            optimizer.zero_grad()
            inputs = interleave(torch.cat((inputs_x, inputs_u_w, inputs_u_s)), ratio*2+1).to(device)
            logits = model(inputs)
            logits = de_interleave(logits, ratio*2+1)
            logits_x = logits[:batch_size]
            logits_u_w, logits_u_s = logits[batch_size:].chunk(2)
            del logits

            # Compute the loss for labeled data
            Lx = criterion(logits_x, targets_x)
            loss_s += Lx.item() * inputs_x.size(0)

            # Compute the pseudo-labels and loss for unlabeled data
            probs = torch.softmax(logits_u_w, dim=-1)
            max_probs, targets_u = torch.max(probs, dim=-1)
            mask = max_probs.ge(threshold).bool()

            Lu = (F.cross_entropy(logits_u_s, targets_u, reduction='none') * mask).mean()

            loss_u += Lu.item() * mask.sum()

            # Total loss
            loss = Lx + lambda_u * Lu

            # Backward and optimize
            loss.backward()
            optimizer.step()
            model.zero_grad()

        loss_s = loss_s / (eval_step * batch_size)
        loss_u = loss_u / (eval_step * batch_size * ratio)

        # print(f'Supervised Loss: {loss_s}, Unsupervised Loss: {loss_u}')

        # Validation phase
        model.eval()  # Set model to evaluate mode
        val_loss = 0.0
        val_corrects = 0
        
        with torch.no_grad():
            for inputs_w, inputs_s, labels in val_loader:
                inputs = inputs_w.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(test_subset)
        val_acc = val_corrects.double() / len(test_subset)
        # print(f'Validation Loss: {val_loss}, Acc: {val_acc}')
        if val_acc > best_acc:
            best_acc = val_acc
    
    return best_acc

In [21]:
def active_sampling(model, unlabeled_subset, strategy = 'uncertainty', forward_passes = 100):
    model.eval()
    with torch.no_grad():
        all_predictions = []
        unlabeled_loader = DataLoader(unlabeled_subset, batch_size=batch_size, shuffle=False, num_workers=2)
        if strategy == 'uncertainty':
            for _, inputs_s, _ in unlabeled_loader:
                mean_predictions = []
                for _ in range(forward_passes):
                    inputs = inputs_s.to(device)
                    outputs = model(inputs)
                    probs = torch.softmax(outputs, dim=-1)
                    mean_predictions.append(probs.unsqueeze(0))
                mean_predictions = torch.cat(mean_predictions)
                mean_predictions = torch.mean(mean_predictions, 0)
                all_predictions.append(mean_predictions)
            all_predictions = torch.cat(all_predictions)

            safe_probabilities = all_predictions.clamp(min=1e-9)
            scores = -torch.sum(safe_probabilities * torch.log(safe_probabilities), dim=1)
            _, indices = torch.topk(scores, batch_size)
        elif strategy == 'diversity':
            for inputs_w, _, _ in unlabeled_loader:
                mean_predictions = []
                for _ in range(forward_passes):
                    inputs = inputs_s.to(device)
                    outputs = model(inputs)
                    probs = torch.softmax(outputs, dim=-1)
                    mean_predictions.append(probs.unsqueeze(0))
                mean_predictions = torch.cat(mean_predictions)
                mean_predictions = torch.mean(mean_predictions, 0)
                all_predictions.append(mean_predictions)
            all_predictions = torch.cat(all_predictions)
            max_probs, targets_u = torch.max(all_predictions, dim=-1)
            mask = max_probs.ge(threshold).bool()
            
        else:
            pass
    
    return indices
                

In [36]:
def active_sampling(model, unlabeled_subset, strategy = 'uncertainty', forward_passes = 100):
    model.eval()
    with torch.no_grad():
        unlabeled_loader = DataLoader(unlabeled_subset, batch_size=batch_size, shuffle=False, num_workers=2)
        if strategy == 'uncertainty':
            all_predictions = []
            for _, inputs_s, _ in unlabeled_loader:
                mean_predictions = []
                for _ in range(forward_passes):
                    inputs = inputs_s.to(device)
                    outputs = model(inputs)
                    probs = torch.softmax(outputs, dim=-1)
                    mean_predictions.append(probs.unsqueeze(0))
                mean_predictions = torch.cat(mean_predictions)
                mean_predictions = torch.mean(mean_predictions, 0)
                all_predictions.append(mean_predictions)
            all_predictions = torch.cat(all_predictions)

            safe_probabilities = all_predictions.clamp(min=1e-9)
            scores = -torch.sum(safe_probabilities * torch.log(safe_probabilities), dim=1)
            _, indices = torch.topk(scores, batch_size)
        elif strategy == 'diversity':
            # Compute the pairwise similarity between predictions
            for inputs_w, _, _ in unlabeled_loader:
                mean_predictions = []
                for _ in range(forward_passes):
                    inputs = inputs_s.to(device)
                    outputs = model(inputs)
                    probs = torch.softmax(outputs, dim=-1)
                    mean_predictions.append(probs.unsqueeze(0))
                mean_predictions = torch.cat(mean_predictions)
                mean_predictions = torch.mean(mean_predictions, 0)
                all_predictions.append(mean_predictions)
            all_predictions = torch.cat(all_predictions)
            # Compute the pairwise similarity between predictions
            similarity_matrix = torch.zeros((len(all_predictions), len(all_predictions)))
            for i in range(len(all_predictions)):
                for j in range(i+1, len(all_predictions)):
                    similarity_matrix[i, j] = torch.cosine_similarity(all_predictions[i], all_predictions[j])
                    similarity_matrix[j, i] = similarity_matrix[i, j]
            
            # Select the indices with the highest diversity
            selected_indices = []
            while len(selected_indices) < batch_size:
                # Find the index with the lowest average similarity to the already selected indices
                min_similarity = float('inf')
                min_index = None
                for i in range(len(all_predictions)):
                    if i not in selected_indices:
                        similarity = torch.mean(similarity_matrix[i, selected_indices])
                        if similarity < min_similarity:
                            min_similarity = similarity
                            min_index = i
                
                # Add the selected index to the list and update the similarity matrix
                selected_indices.append(min_index)
                similarity_matrix = torch.cat((similarity_matrix[:min_index, :], similarity_matrix[min_index+1:, :]), dim=0)
                similarity_matrix = torch.cat((similarity_matrix[:, :min_index], similarity_matrix[:, min_index+1:]), dim=1)
        elif strategy == 'consistency':
            all_predictions_s = []
            all_predictions_w = []
            for inputs_w, inputs_s, _ in unlabeled_loader:
                mean_predictions_w = []
                mean_predictions_s = []
                for _ in range(forward_passes):
                    inputs_s = inputs_s.to(device)
                    inputs_w = inputs_w.to(device)
                    outputs_s = model(inputs_s)
                    outputs_w = model(inputs_w)
                    probs_s = torch.softmax(outputs_s, dim=-1)
                    probs_w = torch.softmax(outputs_w, dim=-1)
                    mean_predictions_s.append(probs_s.unsqueeze(0))
                    mean_predictions_w.append(probs_w.unsqueeze(0))
                mean_predictions_s = torch.cat(mean_predictions_s)
                mean_predictions_s = torch.mean(mean_predictions_s, 0)
                mean_predictions_w = torch.cat(mean_predictions_w)
                mean_predictions_w = torch.mean(mean_predictions_w, 0)
                
                all_predictions_s.append(mean_predictions_s)
                all_predictions_w.append(mean_predictions_w)
            all_predictions_s = torch.cat(all_predictions_s)
            all_predictions_w = torch.cat(all_predictions_w)
            # Compute the pairwise similarity between predictions
            scores = -torch.cosine_similarity(all_predictions_s, all_predictions_w, dim = -1)
            _, indices = torch.topk(scores, batch_size)
        elif strategy == 'combined':
            all_predictions_s = []
            all_predictions_w = []
            for inputs_w, inputs_s, _ in unlabeled_loader:
                mean_predictions_w = []
                mean_predictions_s = []
                for _ in range(forward_passes):
                    inputs_s = inputs_s.to(device)
                    inputs_w = inputs_w.to(device)
                    outputs_s = model(inputs_s)
                    outputs_w = model(inputs_w)
                    probs_s = torch.softmax(outputs_s, dim=-1)
                    probs_w = torch.softmax(outputs_w, dim=-1)
                    mean_predictions_s.append(probs_s.unsqueeze(0))
                    mean_predictions_w.append(probs_w.unsqueeze(0))
                mean_predictions_s = torch.cat(mean_predictions_s)
                mean_predictions_s = torch.mean(mean_predictions_s, 0)
                mean_predictions_w = torch.cat(mean_predictions_w)
                mean_predictions_w = torch.mean(mean_predictions_w, 0)
                
                all_predictions_s.append(mean_predictions_s)
                all_predictions_w.append(mean_predictions_w)
            all_predictions_s = torch.cat(all_predictions_s)
            all_predictions_w = torch.cat(all_predictions_w)
            # Compute the pairwise similarity between predictions
            consistency_scores = torch.cosine_similarity(all_predictions_s, all_predictions_w, dim = -1)
            safe_probabilities = all_predictions_s.clamp(min=1e-9)
            uncertainty_scores = -torch.sum(safe_probabilities * torch.log(safe_probabilities), dim = 1)
            combined_scores = consistency_scores * uncertainty_scores
            _, indices = torch.topk(combined_scores, batch_size)
        else:
            pass
    
    return indices
                

In [40]:
for i in range(5, 10):
    set_seed(seed)
    model = CRCClassifier().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    curr_train_data_portion = 0.01*i
    print(f'Currently {curr_train_data_portion*100}% of the train data is being used')
    if i == 5:
        unlabeled_subset, labeled_subset = stratified_split(train_subset, test_size=curr_train_data_portion)
        labeled_loader = DataLoader(labeled_subset, batch_size=batch_size, shuffle=True, num_workers=2)
        unlabeled_loader = DataLoader(unlabeled_subset, batch_size=batch_size*ratio, shuffle=True, num_workers=2)
        acc = train_fixmatch_multistep(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1)
        #torch.save(model.state_dict(), 'semi_5_percents_combined.pth')
        indices = active_sampling(model, unlabeled_subset, 'combined')
        labeled_subset = ConcatDataset([labeled_subset,Subset(unlabeled_subset, indices.tolist())])
        all_indices = torch.arange(len(unlabeled_subset)).to(device)
        mask = ~torch.isin(all_indices, indices)
        indices_to_keep = all_indices[mask]
        unlabeled_subset = Subset(dataset, indices_to_keep.tolist())
    else:
        labeled_loader = DataLoader(labeled_subset, batch_size=batch_size, shuffle=True, num_workers=2)
        unlabeled_loader = DataLoader(unlabeled_subset, batch_size=batch_size*ratio, shuffle=True, num_workers=2)
        acc = train_fixmatch_multistep(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1)
        indices = active_sampling(model, unlabeled_subset, 'combined')
        labeled_subset = ConcatDataset([labeled_subset,Subset(unlabeled_subset, indices.tolist())])
        all_indices = torch.arange(len(unlabeled_subset)).to(device)
        mask = ~torch.isin(all_indices, indices)
        indices_to_keep = all_indices[mask]
        unlabeled_subset = Subset(dataset, indices_to_keep.tolist())
    print(f'best_acc:{acc}')
    with open('semi_val_accs_act_combined_0.05_3.txt','a+') as f:
        f.write(f'{acc}\n')
    f.close()

Currently 5.0% of the train data is being used
Supervised Loss: 0.9058436810970306, Unsupervised Loss: 0.02819581888616085
Validation Loss: 1.3242982771992684, Acc: 0.5760000000000001
Supervised Loss: 1.2299780127406121, Unsupervised Loss: 0.0011561702704057097
Validation Loss: 0.9649851381778717, Acc: 0.554
Supervised Loss: 0.7552391138672828, Unsupervised Loss: 0.0013084320817142725
Validation Loss: 0.9268070340156556, Acc: 0.649
Supervised Loss: 0.5517346034944057, Unsupervised Loss: 0.01132572628557682
Validation Loss: 0.8495236402750015, Acc: 0.736
Supervised Loss: 0.5128815933316946, Unsupervised Loss: 0.020450729876756668
Validation Loss: 0.8743248873949051, Acc: 0.735
Supervised Loss: 0.36257463522255423, Unsupervised Loss: 0.042530592530965805
Validation Loss: 0.8804135692119598, Acc: 0.745
Supervised Loss: 0.24684018716216088, Unsupervised Loss: 0.08287295699119568
Validation Loss: 1.0889705610275269, Acc: 0.734
Supervised Loss: 0.17930127911269664, Unsupervised Loss: 0.12142

In [37]:
active_sampling(model, unlabeled_subset, 'consistency')

tensor([1031, 1769, 1320,  328, 1542,  261, 1138,  676, 1102, 1325, 1861, 1335,
        2445, 1459, 1880, 1434, 1692, 1717,  304,   81,  400, 1748, 1255,  468,
        1609, 1505, 1354, 1697,  246, 1867, 1132, 1148,  568,  311, 1254,  969,
        1016, 1204, 1078,  611], device='cuda:0')

In [32]:
for i in range(5, 10):
    model = CRCClassifier().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    curr_train_data_portion = 0.01*i
    print(f'Currently {curr_train_data_portion*100}% of the train data is being used')
    if i == 5:
        unlabeled_subset, labeled_subset = stratified_split(train_subset, test_size=curr_train_data_portion)
        labeled_loader = DataLoader(labeled_subset, batch_size=batch_size, shuffle=True, num_workers=2)
        unlabeled_loader = DataLoader(unlabeled_subset, batch_size=batch_size*ratio, shuffle=True, num_workers=2)
        acc = train_fixmatch_multistep(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1)
        torch.save(model.state_dict(), 'semi_5_percents.pth')
        indices = active_sampling(model, unlabeled_subset)
        labeled_subset = ConcatDataset([labeled_subset,Subset(unlabeled_subset, indices.tolist())])
        all_indices = torch.arange(len(unlabeled_subset)).to(device)
        mask = ~torch.isin(all_indices, indices)
        indices_to_keep = all_indices[mask]
        unlabeled_subset = Subset(dataset, indices_to_keep.tolist())
    else:
        labeled_loader = DataLoader(labeled_subset, batch_size=batch_size, shuffle=True, num_workers=2)
        unlabeled_loader = DataLoader(unlabeled_subset, batch_size=batch_size*ratio, shuffle=True, num_workers=2)
        acc = train_fixmatch_multistep(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1)
        indices = active_sampling(model, unlabeled_subset)
        labeled_subset = ConcatDataset([labeled_subset,Subset(unlabeled_subset, indices.tolist())])
        all_indices = torch.arange(len(unlabeled_subset)).to(device)
        mask = ~torch.isin(all_indices, indices)
        indices_to_keep = all_indices[mask]
        unlabeled_subset = Subset(dataset, indices_to_keep.tolist())
    print(f'best_acc:{acc}')
    with open('semi_val_accs_act_0.05.txt','a+') as f:
        f.write(f'{acc}\n')
    f.close()

Currently 5.0% of the train data is being used


KeyboardInterrupt: 

In [39]:
start_percentage = 0.01
for i in range(41,51):
    model = CRCClassifier().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    curr_train_data_portion = start_percentage*i
    print(f'Currently {curr_train_data_portion*100}% of the train data is being used')
    curr_train_data_len = int(len(train_subset)*curr_train_data_portion)
    indices = torch.randperm(len(train_subset))[:curr_train_data_len]
    curr_train_data = Subset(train_subset, indices)
    train_loader = DataLoader(curr_train_data, batch_size=40, shuffle=True, num_workers=2)
    val_loader = DataLoader(test_subset, batch_size=40, shuffle=False)
    acc = train_fixmatch_multistep(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1)
    print(f'best_acc:{acc}')
    with open('semi_val_accs_stratified_1.txt','a+') as f:
        f.write(f'{acc}\n')
    f.close()

Currently 41.0% of the train data is being used
Supervised Loss: 1.2811084246635438, Unsupervised Loss: 0.006343635264784098
Validation Loss: 3.032578038573265, Acc: 0.387
Supervised Loss: 1.1823661908507348, Unsupervised Loss: 0.0006406180909834802
Validation Loss: 0.9089127492904663, Acc: 0.579
Supervised Loss: 0.7857964646816253, Unsupervised Loss: 0.0019456836162135005
Validation Loss: 0.6994425225257873, Acc: 0.738


KeyboardInterrupt: 

In [19]:
train_fixmatch(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1)

Supervised Loss: 0.8793807696551085, Unsupervised Loss: 0.022424958646297455
Validation Loss: 1.616305254875806, Acc: 0.581
Supervised Loss: 1.4238559740781784, Unsupervised Loss: 0.00122667639516294
Validation Loss: 0.8330651044845581, Acc: 0.68
Supervised Loss: 0.8244266611337662, Unsupervised Loss: 0.00022006094513926655
Validation Loss: 0.740202261209488, Acc: 0.715
Supervised Loss: 0.7158406937122345, Unsupervised Loss: 0.002668942091986537
Validation Loss: 1.06941114962101, Acc: 0.6
Supervised Loss: 0.6107849794626236, Unsupervised Loss: 0.007886268198490143
Validation Loss: 0.7874435633420944, Acc: 0.722
Supervised Loss: 0.5477163337171078, Unsupervised Loss: 0.013094935566186905
Validation Loss: 0.7109185874462127, Acc: 0.731
Supervised Loss: 0.4829804074764252, Unsupervised Loss: 0.026563582941889763
Validation Loss: 0.7224354898929596, Acc: 0.75
Supervised Loss: 0.42736384078860284, Unsupervised Loss: 0.039599817246198654
Validation Loss: 0.7682253640890121, Acc: 0.768
Superv

KeyboardInterrupt: 

In [ ]:
train_fixmatch(model, labeled_loader, unlabeled_loader, optimizer, num_epochs, eval_step, threshold, lambda_u=1)

Supervised Loss: 1.0134291757643223, Unsupervised Loss: 0.025716563686728477
Validation Loss: 1.1932102400064468, Acc: 0.667
Supervised Loss: 1.311743875145912, Unsupervised Loss: 0.001087491400539875
Validation Loss: 0.9457165050506592, Acc: 0.587
Supervised Loss: 0.7710262385010719, Unsupervised Loss: 0.001857718569226563
Validation Loss: 1.0982925128936767, Acc: 0.539
Supervised Loss: 0.639760956466198, Unsupervised Loss: 0.008201740682125092
Validation Loss: 0.7381910591199994, Acc: 0.721
Supervised Loss: 0.4721998928487301, Unsupervised Loss: 0.03007802926003933
Validation Loss: 0.8140553459525108, Acc: 0.714
Supervised Loss: 0.34529198694974184, Unsupervised Loss: 0.062282633036375046
Validation Loss: 0.9334692762233316, Acc: 0.705
Supervised Loss: 0.24022329680621624, Unsupervised Loss: 0.09683683514595032
Validation Loss: 0.8809191972389817, Acc: 0.761
Supervised Loss: 0.18249399345368147, Unsupervised Loss: 0.12416637688875198
Validation Loss: 0.956484092772007, Acc: 0.759
Sup

In [39]:
def interleave(x, size):
    s = list(x.shape)
    return x.reshape([-1, size] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])


def de_interleave(x, size):
    s = list(x.shape)
    return x.reshape([size, -1] + s[1:]).transpose(0, 1).reshape([-1] + s[1:])

In [46]:
from torchsummary import summary
summary(model, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 75, 75]           9,408
       BatchNorm2d-2           [-1, 64, 75, 75]             128
              ReLU-3           [-1, 64, 75, 75]               0
         MaxPool2d-4           [-1, 64, 38, 38]               0
            Conv2d-5           [-1, 64, 38, 38]          36,864
       BatchNorm2d-6           [-1, 64, 38, 38]             128
              ReLU-7           [-1, 64, 38, 38]               0
            Conv2d-8           [-1, 64, 38, 38]          36,864
       BatchNorm2d-9           [-1, 64, 38, 38]             128
             ReLU-10           [-1, 64, 38, 38]               0
       BasicBlock-11           [-1, 64, 38, 38]               0
           Conv2d-12           [-1, 64, 38, 38]          36,864
      BatchNorm2d-13           [-1, 64, 38, 38]             128
             ReLU-14           [-1, 64,